You can find this ipynb also in github, on

# Import Libraries

In [2]:
# Here we import the libraries we will use
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf

def install_agent_libraries():
  !pip install swig
  !pip install gymnasium[box2d]
  !pip install ray[rllib]
  !pip install renderlab
#install_agent_libraries()

In [3]:
# Check if GPU is enabled
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f46e924d-811e-6149-4d51-80f296d971cd)


# 1. Express the moves (rock, scissors, paper) into (0, 1, 2) respectively

# 2. Split the dataset into train-test: You can for each class select a ratio as test set (e.g. 20% of rock, 20% of scissors and 20% of paper) and use these to test your model/agent.

# 3. Select Image: Select a radom image frm the 2100 (which corresponds to move 0, or 1, or 2)

# 4. Preprocess Image: Process the image:
*  a. wih probability $p_1$ apply Vertical flip
*  b. with probability $p_2$ apply Horizontal flip
*  c. Add noise with $\mu = 0$, $\sigma = 255\cdot 0.05$. If you normalize the image, adjust standard deviation accordingly.
*  d. You can apply any other method Image Processing you want, so that you can make the game harder (if you want).


# 5. Apply: Send the image to your agent

# 6. The agent read the image and selects the optimal action

# 7. The goal is maximizing gain. So, you will need to plot the agent's gain (You can store the total gain for every round and plot it at the end of the game)

# 8. As end goal, try the accuracy of the agent (or model) on images outside of the dataset, e.g. from the intenret or your own, which you must rescale to the same size. For example, you can try the action it will output for the attached image

# 5. Create an agent. Use summart to print the policy model. Describe the created Neural Network.

In [5]:
def import_agent():
  import gymnasium as gym
  import ray
  from ray.rllib.algorithms.ppo import PPOConfig
#import_agent()

In [6]:
def agent_definition_01():
  ray.shutdown()
  ray.init()

  config = PPOConfig()
  config.use_critic = True # Use Actor-Critic Architecture
  config.use_gae = True # Use Generalzed Advantage Updates
  config.lambda_ = 0.95 # Lambda Learning parameter
  config.user_kl_loss = True # Use KL-Penalty in addition to e-Clipping
  config.sgd_minibatch_size = 32 # Batch Size
  config.num_sgd_iter = 30 # Epochs
  config.shuffle_sequences = True # Whether to shffle episode sequences
  config.vf_loss_coeff = 0.5 # Value Function loss Regularization
  config.entropy_coeff = 0.001 # Entropy Regularization coefficient for Exploration
  config.clip_param = 0.2 # e-Clipping parameter

  agent = config.environment("LunarLander-v2").framework(framework='tf').build()
  agent.get_policy().model.base_model.summary(expand_nested=True)
#agent_definition_01()

# 6. Create a train loop to train the agent

In [8]:
def agent_train():
  train_iterations = 100
  average_rewards_per_iteration = []

  for i in range(train_iterations):
    log = agent.train()
    average_rewards = log['sampler_results']['episode_reward_mean']
    average_rewards_per_iteration.append(average_rewards)

    print(f'Iteration {i + 1}, Average Rewards: {average_rewards}')


  plt.plot(average_rewards_per_iteration)
  plt.title('PPO LunarLander-v2')
  plt.xlabel('Iterations')
  plt.ylabel('Average Rewards')
  plt.show()
#agent_train()

# 7. Use the library renderlab to "render" the gym environmonet in colab, to see how the trained agent plays.

In [9]:
#import renderlab as rl

In [10]:
def agent_run():
  env = gym.make("LunarLander-v2", render_mode = "rgb_array")
  env = rl.RenderFrame(env, "./output")

  observation, info = env.reset()

  while True:
    action = env.action_space.sample()
    observation,reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
      break

  env.play()
#agent_run()